# Downloading all `.fastq`s in a project

In this vignette, we'll demonstrate how to identify bundles associated with a given project, and how to download all `.fastq` files in those bundles.

This vignette assumes some familiarity with the HCA API and command line tools. If you need to brush up, try paging through [the other vignettes](https://github.com/HumanCellAtlas/data-consumer-vignettes/) - the [Download Any BAM File](https://github.com/HumanCellAtlas/data-consumer-vignettes/tree/master/Download%20Any%20BAM%20File) vignette will be particularly relevant. Make sure that you've installed the HCA CLI and that you're logged into it before starting!

## Method 1: Using the Data Explorer

The fastest way to get all `.fastq`s associated with a project is to find the project in the HCA Data Explorer, then download a file manifest for the `.fastq`s associated with the project, then use `hca dss download-manifest` to download the files you selected. See [this guide](https://data.humancellatlas.org/guides/quick-start-guidehttps://data.humancellatlas.org/guides/quick-start-guide) for details.

## Method 2: Using the HCA DSS REST API, the long way

We can also retrieve all `.fastq`s associated with a project with the DSS API. First, you'll want to find a project that you like on the [HCA Data Explorer](https://data.humancellatlas.org/explore/). (You'll probably want to filter for `.fastq` files to make sure that whatever you're downloading has the kind of data that you're looking for.)

Once you find a project you like, open the project in the Data Explorer and then copy its URL. The UUID is at the end of the URL:

In [1]:
url = 'https://data.humancellatlas.org/explore/projects/74b6d569-3b11-42ef-b6b1-a0454522b4a0'
_, project_uuid = url.rsplit('/', maxsplit=1)
project_uuid

'74b6d569-3b11-42ef-b6b1-a0454522b4a0'

(This tutorial works a lot better if you choose a project that you know has `.fastq`s in it - see Method 1).

Given a project UUID, we can make a `POST /search` request to the Data Store to search for all bundles that belong to the given project. `POST /search` requests accept ElasticSearch queries. In a bundle, the `files.project_json.provenance.document_id` field contains the project UUID. That means we want a query that looks like this:

    {"query": {"bool": {"must": {"match": {"files.project_json.provenance.document_id": project_uuid}}}}}
    
We can use the HCA API to search for bundles matching this query.

In [2]:
import hca.dss
client = hca.dss.DSSClient()
query = {'query': {'bool': {'must': {'match': {'files.project_json.provenance.document_id': project_uuid}}}}}
results = client.post_search(replica='aws', es_query=query)
bundle_uuid, bundle_version = results['results'][0]['bundle_fqid'].split('.', maxsplit=1)
results['results'][0]

{'bundle_fqid': 'fffcea5e-2e6c-4ca1-9aa9-c23b90b2e8b8.2019-05-16T211813.059000Z',
 'bundle_url': 'https://dss.data.humancellatlas.org/v1/bundles/fffcea5e-2e6c-4ca1-9aa9-c23b90b2e8b8?version=2019-05-16T211813.059000Z&replica=aws',
 'search_score': None}

We can actually improve this code - `post_search` only returns so many results at one time. If we use the `post_search.iterate` method, the API Python bindings will handle pagination for us transparently:

In [3]:
results = client.post_search.iterate(replica='aws', es_query=query)
for result in results:
    bundle_uuid, bundle_version = result['bundle_fqid'].split('.', maxsplit=1)
    print(bundle_uuid, bundle_version)

fffcea5e-2e6c-4ca1-9aa9-c23b90b2e8b8 2019-05-16T211813.059000Z
ffc1714d-caef-41fe-9b68-a6db5efd4c69 2019-05-16T211813.060000Z
ffa728d1-3767-46a0-ad94-adefd36630c9 2019-05-16T211813.105000Z
ff982627-9abf-4d66-bc45-f9298ffb9311 2019-05-16T211813.111000Z
ff8020e9-5bce-491b-b5f3-821ab25fb167 2019-05-16T211813.082000Z
ff781a9d-e4f1-4a4f-acee-8bf28d8cd684 2019-05-16T211813.068000Z
ff74ec2f-21b7-4949-8c55-42315afa8295 2019-05-16T211813.074000Z
ff5e0cfc-bb2a-4f20-bade-897f1626fa8d 2019-05-16T211813.074000Z
ff5b1de9-fb65-4451-95dc-7da7d11bd493 2019-05-16T211813.105000Z
ff5a4036-08cf-4abc-98a7-572eaaa452cc 2019-05-16T211813.057000Z
ff3d4b8d-11c9-4d63-976f-35be1269be85 2019-05-16T211813.063000Z
ff20ee03-a70a-41aa-a0fc-7208e5b1bcfd 2019-05-16T211813.080000Z
ff1f55b5-bbc0-4bcc-8ea2-b50ac62f35bd 2019-05-16T211813.065000Z
ff1c2885-f944-47b3-9564-a4c759fa52af 2019-05-16T211813.080000Z
ff18cc1a-7397-455f-a9be-4f2c04b70094 2019-05-16T211813.063000Z
ff137301-5bf0-4e64-8cac-a563814101d7 2019-05-16T211813.

f58709df-53ec-4e19-8218-adb07a13404a 2019-05-16T211813.077000Z
f583a3a9-81b3-45fb-bdb0-da61744d4c44 2019-05-16T211813.064000Z
f57755b3-54aa-4fd0-8fba-22fb3ba648db 2019-05-16T211813.111000Z
f55eaa5d-f7c3-46e7-bca8-98b16cb411c2 2019-05-16T211813.101000Z
f55712ce-6dff-435b-ab00-fc88d88e0502 2019-05-16T211813.105000Z
f53dfc8e-c590-4e03-927d-0cb93f43c559 2019-05-16T211813.105000Z
f52cba8a-1f62-4a63-b6a0-b62797c1c963 2019-05-16T211813.063000Z
f529a2db-e977-4515-a3e0-563218d8897e 2019-05-16T211813.073000Z
f528ba6e-fa46-4412-9265-ad3155cc1208 2019-05-16T211813.058000Z
f527257f-7e68-4e51-8ec4-e73a07a811ac 2019-05-16T211813.076000Z
f51c5ae8-0b3a-4aac-bba0-ae056af83d53 2019-05-16T211813.063000Z
f519fc44-1432-409a-894d-7a796bc8acf6 2019-05-16T211813.065000Z
f50c97f8-2e87-42a3-b04e-68713b3f014a 2019-05-16T211813.062000Z
f4fd9233-171a-43a3-8e13-d5f5c02c5548 2019-05-16T211813.064000Z
f4f1180f-12aa-4eb1-b641-dd493412434d 2019-05-16T211813.101000Z
f4ef702e-8138-45de-b0df-2eee6d0c8a4a 2019-05-16T211813.

e7aa43d3-d866-4ade-9b5c-20fa7d59b5c9 2019-05-16T211813.080000Z
e7a5ef44-bf84-4003-a92b-8a5a9e663ac3 2019-05-16T211813.105000Z
e79c84c9-98aa-46b6-a794-dcba115f96a5 2019-05-16T211813.073000Z
e7998f76-1a8d-4381-8e1b-de8d65dd25aa 2019-05-16T211813.057000Z
e775f042-c533-45a1-93be-8ca7be5353c6 2019-05-16T211813.098000Z
e772e207-8c35-4876-b640-e0aac6597620 2019-05-16T211813.080000Z
e76f638e-a9e3-4dcf-92b0-66b717dcd67f 2019-05-16T211813.070000Z
e762b6e7-8f80-4f33-b721-e880a1871564 2019-05-16T211813.097000Z
e7622a86-37a1-4a62-bd47-4c2f0df60f32 2019-05-16T211813.105000Z
e74dd521-4d07-46f5-a2a0-fea393d82364 2019-05-16T211813.077000Z
e74488ec-160b-4e08-b633-68ae726aa1f0 2019-05-16T211813.107000Z
e73ebaf4-be3d-4f48-8c4b-5824122c341f 2019-05-16T211813.086000Z
e73787a3-ba59-4bac-8f95-b10f80ea256e 2019-05-16T211813.073000Z
e72fbb95-254c-493b-af25-d129a2c8b7eb 2019-05-16T211813.087000Z
e7291ec8-1ebc-431e-bde2-ea5e41116d36 2019-05-16T211813.057000Z
e723705b-ec49-434e-9b03-c682af9de60f 2019-05-16T211813.

dea8fc55-4396-473b-947d-979e67186a0f 2019-05-16T211813.107000Z
de8d7853-2b6d-4340-a782-1689291c5637 2019-05-16T211813.076000Z
de7f5690-8a96-4f18-85d1-9b2088b6656f 2019-05-16T211813.082000Z
de7deac7-574b-4528-be70-284ad156ec51 2019-05-16T211813.110000Z
de4c24ef-8c56-4fcd-bcf1-38af1a8121bc 2019-05-16T211813.092000Z
de4a317e-9682-4199-b68a-a235f0581916 2019-05-16T211813.080000Z
de46c4f4-e291-4f02-a042-bb5dd8cf752e 2019-05-16T211813.104000Z
de3e50d0-58a3-4b0e-977d-dc31b36fd473 2019-05-16T211813.081000Z
de3e1f03-a0ce-4953-b697-69b3e17dd474 2019-05-16T211813.061000Z
de2a444d-8745-48fa-874f-a54e1461c8f8 2019-05-16T211813.089000Z
ddfcdbf1-cc32-4459-bc81-83a8a6c8f545 2019-05-16T211813.094000Z
ddf3b807-400c-4cad-ad6e-4274f2b8a7e8 2019-05-16T211813.104000Z
ddd2b65e-2519-44e4-98f4-24448abc02e1 2019-05-16T211813.100000Z
ddd2884e-f977-46bc-82c1-18fdfe878970 2019-05-16T211813.070000Z
ddc71056-6ec9-4fbc-a491-7074b9376382 2019-05-16T211813.081000Z
ddac7390-0f09-4ba2-b6e5-1f04ea0fd079 2019-05-16T211813.

d06c6950-2efb-42f0-a408-23cfe95a6473 2019-05-16T211813.075000Z
d06a0ffb-7600-4191-b28d-64daca95c9e0 2019-05-16T211813.067000Z
d062e7f9-54b7-42df-97d2-c9133fac438c 2019-05-16T211813.084000Z
d0617cdb-d80b-4a6b-8d0f-71806d99c3b4 2019-05-16T211813.098000Z
d0542656-d0e5-45d5-a586-9702f8407243 2019-05-16T211813.108000Z
d03b978f-4979-489b-8a86-6537b4c84521 2019-05-16T211813.066000Z
d03705d0-0e5d-4784-94b0-f91794379978 2019-05-16T211813.077000Z
d0205c0e-e426-4503-9eed-a1a86ffeace6 2019-05-16T211813.084000Z
d00b8f0f-fccc-4600-9781-648651dd0c12 2019-05-16T211813.059000Z
d0073e4a-53e7-43f8-ae67-94fe490569e4 2019-05-16T211813.106000Z
cff530e7-7539-4e93-b217-c9599c804584 2019-05-16T211813.083000Z
cfe89bd5-ccd0-45fb-8f69-02ca099aafd0 2019-05-16T211813.107000Z
cfe61c18-37f3-4f79-8b07-13eccb496827 2019-05-16T211813.074000Z
cfe27fc6-cf96-471f-af6f-0dfa6edf0aec 2019-05-16T211813.110000Z
cfdcde80-0745-46e0-80ab-0d366033b030 2019-05-16T211813.074000Z
cfd3871b-0471-4914-8437-fc0ea265a5de 2019-05-16T211813.

...

2cf4f246-5a7b-40d9-826f-13005e76a767 2019-05-16T211813.075000Z
2cf1f334-2c9f-4ab2-8927-3b607a4a6ef9 2019-05-16T211813.083000Z
2ce92b18-ca92-4c96-aafe-6281f7765e17 2019-05-16T211813.109000Z
2ce5fbd4-ce23-47b0-b4a9-6c85b0ef5f55 2019-05-16T211813.096000Z
2cdf7231-8b03-4703-abf3-8009770d5344 2019-05-16T211813.076000Z
2cdb2592-c70d-4749-8ab8-26ea677677dd 2019-05-16T211813.071000Z
2cd9479d-d659-48a7-9f58-bb5933a8f608 2019-05-16T211813.066000Z
2cd8156e-a947-4087-8cd0-f2aed61ee3dd 2019-05-16T211813.077000Z
2cd0d674-0a8c-4c95-b94d-0064da5e4ffe 2019-05-16T211813.107000Z
2ccac57b-cb37-45f7-84dd-9f6e3496f870 2019-05-16T211813.065000Z
2c9ea179-7e5c-4795-9b4e-860952b78f90 2019-05-16T211813.104000Z
2c93cf00-4358-4cf4-90cd-3036b4825363 2019-05-16T211813.087000Z
2c8c9297-a404-4536-8062-6d42fe466e01 2019-05-16T211813.070000Z
2c879e87-66c5-41f3-bdf7-3a12ea007ef2 2019-05-16T211813.076000Z
2c7fdf1e-dc63-40a1-961c-47ecb5e1bfaf 2019-05-16T211813.071000Z
2c7bd574-9e2e-465e-8704-db0153926786 2019-05-16T211813.

21f5a79d-ddbf-4e90-a7b8-fac8b4b00592 2019-05-16T211813.105000Z
21e31844-2d55-4321-8cad-38709df3b2f7 2019-05-16T211813.067000Z
21e0f2fd-9fd3-40d9-8ed2-a4b4b6bac173 2019-05-16T211813.093000Z
21d7fa7f-7523-4489-9c7e-086c461a03da 2019-05-16T211813.063000Z
21b17ad5-185f-4d01-b45c-f5adc0f2d245 2019-05-16T211813.098000Z
21a85c27-6e9f-411b-bdce-9fc19ac67690 2019-05-16T211813.062000Z
21a5bb95-e750-4be2-b7cc-ac5e65edd9c5 2019-05-16T211813.077000Z
217cf2fe-bb24-49dd-93f5-57cf60f6e4d3 2019-05-16T211813.107000Z
217b0d2b-70d2-4180-970c-e7c90aa2522b 2019-05-16T211813.079000Z
2155ed48-b70d-4ad5-86e6-dc0f01398105 2019-05-16T211813.082000Z
2150fd0b-654e-472c-8857-c06f1024a2cf 2019-05-16T211813.063000Z
2149c83b-e975-4439-9c9e-8259e235a0da 2019-05-16T211813.100000Z
21496004-947f-45c2-be97-5025d64ad69a 2019-05-16T211813.076000Z
2145aa14-14df-4ea9-ae95-e430345373c7 2019-05-16T211813.096000Z
21391307-496f-4de2-9aa3-5a8fb7ade7f6 2019-05-16T211813.059000Z
2124c2b6-dfe6-4584-a1b1-4f8094d0bfea 2019-05-16T211813.

14d055b0-bd20-4f97-bb68-7c303c1edd38 2019-05-16T211813.069000Z
14aefa33-0649-4fb0-ab9c-5b34cbf59236 2019-05-16T211813.074000Z
14aaf19f-afa5-45ca-b028-67cffb988e39 2019-05-16T211813.096000Z
148e5d77-2c5d-49ea-8633-4f63709d1db0 2019-05-16T211813.102000Z
147fd6b9-0840-465d-916c-30351b1cfd73 2019-05-16T211813.094000Z
147ee2d3-e714-43ef-8445-f6228add9952 2019-05-16T211813.077000Z
147ac7e9-4121-4a2e-97c6-ee72f7e8406b 2019-05-16T211813.097000Z
1468c122-32a6-48a2-bf12-443a20f1887b 2019-05-16T211813.058000Z
14625b1e-ed87-49b6-b9a2-b6f6e1e7526d 2019-05-16T211813.077000Z
145f2eeb-db2e-49e3-aeb0-7917a820b497 2019-05-16T211813.087000Z
145c4897-5016-4616-94f2-40fb65a283c8 2019-05-16T211813.104000Z
14586dd3-679e-4b0c-b41d-9dc5e8e83b9c 2019-05-16T211813.065000Z
1452cd7d-233c-417a-af9e-0ed0eddbd3a8 2019-05-16T211813.060000Z
1443314d-83be-4664-b9c1-7493e1720fa2 2019-05-16T211813.081000Z
14406ebc-8f21-42a9-9db1-67a989c60df7 2019-05-16T211813.090000Z
143f03d0-28c3-44f6-90f6-bfdd20c3c4bf 2019-05-16T211813.

0b75a2c6-777d-48ec-a441-679ec7962945 2019-05-16T211813.100000Z
0b68e862-72fb-47da-96ce-cff37218e729 2019-05-16T211813.098000Z
0b652391-60d4-4938-af62-21b694e8d0e7 2019-05-16T211813.101000Z
0b5fc125-e8f6-4fd9-9805-202c9b2ab8a7 2019-05-16T211813.085000Z
0b576f53-1c8d-4c0e-b3db-966faa141187 2019-05-16T211813.093000Z
0b4c7fc0-b06d-4082-a471-f287829f502d 2019-05-16T211813.064000Z
0b4405dd-ac1f-411e-ad73-bcdcd17e06a0 2019-05-16T211813.075000Z
0b416292-5b85-4dff-b061-2ddc93baf51b 2019-05-16T211813.084000Z
0b40891e-b08c-4d44-a19e-1fd918c3d844 2019-05-16T211813.085000Z
0b3b7451-7d38-4df7-96aa-d3907b3b0b1d 2019-05-16T211813.066000Z
0b38a079-b8a2-4033-9a50-7f6c6af66516 2019-05-16T211813.084000Z
0b21fb0a-931f-4152-aebd-5d9bb55b529a 2019-05-16T211813.070000Z
0b1a8b80-43a2-4c2d-a249-a707b2cc51d4 2019-05-16T211813.082000Z
0b098f5f-bbb2-41bb-83bf-ed6e617a854e 2019-05-16T211813.079000Z
0b078b39-5d67-4d01-99a4-a9658f534dd1 2019-05-16T211813.097000Z
0b04985b-6027-4982-ab08-6af1d8e0900b 2019-05-16T211813.

02811aee-f230-46b8-bf0b-1ec44653c4b5 2019-05-16T211813.107000Z
0262960d-21dd-4329-a810-b5341943be69 2019-05-16T211813.079000Z
0260261d-4637-4239-82eb-a53973819ab3 2019-05-16T211813.067000Z
0252d53a-002e-4bb3-bd12-cebe2854c455 2019-05-16T211813.090000Z
023b6946-1b49-4093-8672-2dbfc83ca595 2019-05-16T211813.090000Z
023948e8-11ac-4aef-b470-fa981f527530 2019-05-16T211813.076000Z
0221bf5b-fbfd-4d95-96a7-808ae4862249 2019-05-16T211813.101000Z
021e210b-f613-4c1e-a466-4509c020dc44 2019-05-16T211813.077000Z
021dc149-ff64-44da-9e9d-992103dd8832 2019-05-16T211813.070000Z
0213517e-11bc-4316-9cc7-813607aea8f4 2019-05-16T211813.070000Z
021033b5-1cf9-43ab-86a3-2cfd0970c845 2019-05-16T211813.097000Z
020eccbd-9d9a-49b3-b678-54a9ff6643c4 2019-05-16T211813.109000Z
0206616f-4a9b-4666-a0c4-a9648ca10225 2019-05-16T211813.060000Z
01f0c3f7-b445-4ca3-b2f8-c8fb55f0f6e0 2019-05-16T211813.069000Z
01e27230-a60c-4ac4-aa6e-2b3fec9f8daa 2019-05-16T211813.097000Z
01dc2159-4820-45b0-99e8-8db7a7f257c0 2019-05-16T211813.

Nice! That's a lot of bundles...

Now that we've identified all the bundles belonging to the project we're interested in, we can make a `GET /bundle/{uuid}` request to list all the files in each bundle, and from there, identify all `.fastq` files in each bundle.

In [4]:
client.get_bundle(uuid=bundle_uuid, replica='aws')

{'bundle': {'creator_uid': 8008,
  'files': [{'content-type': 'application/json; dcp-type="metadata/biomaterial"',
    'crc32c': 'c392550c',
    'indexed': True,
    'name': 'cell_suspension_0.json',
    's3_etag': '7d96137869d6e5d46c727f604670db65',
    'sha1': 'c7d6c4cd5c3163925bd6254e18ca5c5b242d73ca',
    'sha256': '8551db210f340cb57c98d2dca014d2893567f8668da0b9cce4c103aef87ef680',
    'size': 1366,
    'uuid': 'c98e9ab2-9b7a-4adf-bf28-8c8406b25cae',
    'version': '2019-05-16T162350.156000Z'},
   {'content-type': 'application/json; dcp-type="metadata/biomaterial"',
    'crc32c': 'd3fc2fae',
    'indexed': True,
    'name': 'specimen_from_organism_0.json',
    's3_etag': '5c2033719c75743ed71058d7becf31ca',
    'sha1': '53b104f207ac57ab1f4e07cf6d5634ca856852e7',
    'sha256': 'cf9325255cc7bc9763844bbd1f50c7616f907985524dffd972d849f430d8bb07',
    'size': 1687,
    'uuid': '324191c3-db02-4671-9c51-1fdca8e6e88f',
    'version': '2019-05-16T161926.507000Z'},
   {'content-type': 'applic

Let's write a function `find_fastqs_in_bundle` that, given a bundle UUID, will return the UUIDs of `.fastq` files in that bundle.

In [5]:
def find_fastqs_in_bundle(bundle_uuid, version):
    bundle = client.get_bundle(uuid=bundle_uuid, version=version, replica='aws')
    fastq_files = []
    for file_ in bundle['bundle']['files']:
        if file_['name'].endswith('.fastq.gz'):
            fastq_files.append(file_)
    return fastq_files

Experienced Pythonistas will recognize that we can make a small improvement to `find_fastqs_in_bundle` that is more concise and a little faster:

In [6]:
def find_fastqs_in_bundle(bundle_uuid,version ):
    bundle = client.get_bundle(uuid=bundle_uuid, version=version, replica='aws')
    return filter(lambda x: x['name'].endswith('.fastq.gz'), bundle['bundle']['files'])

Now, let's try using `find_fastqs_in_bundle` with all of the bundles we found at the beginning of this tutorial:

In [7]:
results = client.post_search(replica='aws', es_query=query)
for result in results['results']:
    bundle_uuid, bundle_version = result['bundle_fqid'].split('.', maxsplit=1)
    for fastq in find_fastqs_in_bundle(bundle_uuid, bundle_version):
        print(f"Bundle {result['bundle_fqid']} has {fastq['name']}")

Bundle fffcea5e-2e6c-4ca1-9aa9-c23b90b2e8b8.2019-05-16T211813.059000Z has E18_20161004_Neurons_Sample_14_S083_L005_I1_002.fastq.gz
Bundle fffcea5e-2e6c-4ca1-9aa9-c23b90b2e8b8.2019-05-16T211813.059000Z has E18_20161004_Neurons_Sample_14_S083_L005_R1_002.fastq.gz
Bundle fffcea5e-2e6c-4ca1-9aa9-c23b90b2e8b8.2019-05-16T211813.059000Z has E18_20161004_Neurons_Sample_14_S083_L005_R2_002.fastq.gz
Bundle ffc1714d-caef-41fe-9b68-a6db5efd4c69.2019-05-16T211813.060000Z has E18_20161004_Neurons_Sample_57_S126_L008_I1_002.fastq.gz
Bundle ffc1714d-caef-41fe-9b68-a6db5efd4c69.2019-05-16T211813.060000Z has E18_20161004_Neurons_Sample_57_S126_L008_R1_002.fastq.gz
Bundle ffc1714d-caef-41fe-9b68-a6db5efd4c69.2019-05-16T211813.060000Z has E18_20161004_Neurons_Sample_57_S126_L008_R2_002.fastq.gz
Bundle ffa728d1-3767-46a0-ad94-adefd36630c9.2019-05-16T211813.105000Z has E18_20160930_Neurons_Sample_61_S058_L002_I1_010.fastq.gz
Bundle ffa728d1-3767-46a0-ad94-adefd36630c9.2019-05-16T211813.105000Z has E18_20160

Bundle fea0535e-3bf0-41d4-9871-10db55c77831.2019-05-16T211813.086000Z has E18_20160930_Neurons_Sample_11_S011_L001_I1_008.fastq.gz
Bundle fea0535e-3bf0-41d4-9871-10db55c77831.2019-05-16T211813.086000Z has E18_20160930_Neurons_Sample_11_S011_L001_R1_008.fastq.gz
Bundle fea0535e-3bf0-41d4-9871-10db55c77831.2019-05-16T211813.086000Z has E18_20160930_Neurons_Sample_11_S011_L001_R2_008.fastq.gz
Bundle fe709d61-cd5d-45fd-bdea-ef51c715120c.2019-05-16T211813.073000Z has E18_20161004_Neurons_Sample_15_S014_L004_I1_005.fastq.gz
Bundle fe709d61-cd5d-45fd-bdea-ef51c715120c.2019-05-16T211813.073000Z has E18_20161004_Neurons_Sample_15_S014_L004_R1_005.fastq.gz
Bundle fe709d61-cd5d-45fd-bdea-ef51c715120c.2019-05-16T211813.073000Z has E18_20161004_Neurons_Sample_15_S014_L004_R2_005.fastq.gz
Bundle fe6b57cc-cfdd-4853-8d5b-77beaa07580a.2019-05-16T211813.079000Z has E18_20160930_Neurons_Sample_50_S048_L006_I1_006.fastq.gz
Bundle fe6b57cc-cfdd-4853-8d5b-77beaa07580a.2019-05-16T211813.079000Z has E18_20160

Bundle fdbb1819-1211-4499-be1c-e5cbefe7c189.2019-05-16T211813.062000Z has E18_20160930_Neurons_Sample_21_S018_L004_I1_001.fastq.gz
Bundle fdbb1819-1211-4499-be1c-e5cbefe7c189.2019-05-16T211813.062000Z has E18_20160930_Neurons_Sample_21_S018_L004_R1_001.fastq.gz
Bundle fdbb1819-1211-4499-be1c-e5cbefe7c189.2019-05-16T211813.062000Z has E18_20160930_Neurons_Sample_21_S018_L004_R2_001.fastq.gz
Bundle fdb3efd3-24fb-4b82-b9b1-4b66410d6c13.2019-05-16T211813.094000Z has E18_20161004_Neurons_Sample_19_S019_L003_I1_009.fastq.gz
Bundle fdb3efd3-24fb-4b82-b9b1-4b66410d6c13.2019-05-16T211813.094000Z has E18_20161004_Neurons_Sample_19_S019_L003_R1_009.fastq.gz
Bundle fdb3efd3-24fb-4b82-b9b1-4b66410d6c13.2019-05-16T211813.094000Z has E18_20161004_Neurons_Sample_19_S019_L003_R2_009.fastq.gz
Bundle fda6b61f-45d8-4565-851b-e85f37b2f479.2019-05-16T211813.068000Z has E18_20161004_Neurons_Sample_57_S117_L001_I1_003.fastq.gz
Bundle fda6b61f-45d8-4565-851b-e85f37b2f479.2019-05-16T211813.068000Z has E18_20161

Bundle fc696b9c-353f-4e57-addf-6261b995a5f3.2019-05-16T211813.076000Z has E18_20161004_Neurons_Sample_62_S061_L002_I1_005.fastq.gz
Bundle fc696b9c-353f-4e57-addf-6261b995a5f3.2019-05-16T211813.076000Z has E18_20161004_Neurons_Sample_62_S061_L002_R1_005.fastq.gz
Bundle fc696b9c-353f-4e57-addf-6261b995a5f3.2019-05-16T211813.076000Z has E18_20161004_Neurons_Sample_62_S061_L002_R2_005.fastq.gz
Bundle fc60492b-5293-43d9-b188-ba1b64d1b86e.2019-05-16T211813.097000Z has E18_20161004_Neurons_Sample_48_S048_L003_I1_009.fastq.gz
Bundle fc60492b-5293-43d9-b188-ba1b64d1b86e.2019-05-16T211813.097000Z has E18_20161004_Neurons_Sample_48_S048_L003_R1_009.fastq.gz
Bundle fc60492b-5293-43d9-b188-ba1b64d1b86e.2019-05-16T211813.097000Z has E18_20161004_Neurons_Sample_48_S048_L003_R2_009.fastq.gz
Bundle fc538017-c36a-4dbf-a94e-998756a9320e.2019-05-16T211813.093000Z has E18_20161004_Neurons_Sample_02_S002_L006_I1_009.fastq.gz
Bundle fc538017-c36a-4dbf-a94e-998756a9320e.2019-05-16T211813.093000Z has E18_20161

Bundle fb58e7de-161e-489a-b097-116e4ce5cb09.2019-05-16T211813.100000Z has E18_20160930_Neurons_Sample_09_S009_L008_I1_010.fastq.gz
Bundle fb58e7de-161e-489a-b097-116e4ce5cb09.2019-05-16T211813.100000Z has E18_20160930_Neurons_Sample_09_S009_L008_R1_010.fastq.gz
Bundle fb58e7de-161e-489a-b097-116e4ce5cb09.2019-05-16T211813.100000Z has E18_20160930_Neurons_Sample_09_S009_L008_R2_010.fastq.gz
Bundle fb5584f9-6d45-4e1c-a9b5-3481c524d8b9.2019-05-16T211813.060000Z has E18_20161004_Neurons_Sample_34_S103_L005_I1_002.fastq.gz
Bundle fb5584f9-6d45-4e1c-a9b5-3481c524d8b9.2019-05-16T211813.060000Z has E18_20161004_Neurons_Sample_34_S103_L005_R1_002.fastq.gz
Bundle fb5584f9-6d45-4e1c-a9b5-3481c524d8b9.2019-05-16T211813.060000Z has E18_20161004_Neurons_Sample_34_S103_L005_R2_002.fastq.gz
Bundle fb483d5e-274f-4cd7-8cca-2dc5a5ae107e.2019-05-16T211813.078000Z has E18_20160930_Neurons_Sample_29_S027_L001_I1_006.fastq.gz
Bundle fb483d5e-274f-4cd7-8cca-2dc5a5ae107e.2019-05-16T211813.078000Z has E18_20160

Note that we used `client.post_search` instead of `client.post_search.iterate`, because this operation could take a long time if there are a lot of bundles.